In [1]:
%cd ..

c:\Users\Daniel\cyanobacteria-monitoring


In [2]:
import yaml
import pandas as pd

from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import PowerTransformer
from sklearn.compose import TransformedTargetRegressor

from sklearn.ensemble import RandomForestRegressor
import xgboost
from sklearn.model_selection import KFold

import plotly.express as px

import numpy as np

import optuna
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_optimization_history

c:\Users\Daniel\cyanobacteria-monitoring\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import requests
from bs4 import BeautifulSoup

url = "https://dados.gov.br/dataset/sisagua-controle-mensal-resultado-de-analises"
data = requests.get(url).text
soup = BeautifulSoup(data, "html.parser")


In [64]:
from lib2to3.pgen2.token import COLONEQUAL


table = soup.find("table", {"class": "table table-striped table-bordered table-condensed"})

row_headers = []
table_values = []

for x in table.find_all("tr"):
    for y in x.find_all("th"):
        if y.text not in ["Campo", "Valor"]:
            row_headers.append(y.text)

for x in table.find_all("tr")[1:]:
    td_tags = x.find_all("td")
    td_text = [y.text for y in td_tags]
    table_values.append(td_text)

resultado = []
for (header, val) in zip(row_headers, table_values):
    resultado.append((header, val[0]))

resultado = pd.DataFrame(resultado, columns=["Campo", "Valor"])

In [84]:
resultado.loc[resultado["Campo"] == "Última Atualização", "Valor"].str.replace("\n", "").str.strip()

3    2 de Setembro de 2022, 02:54 (BRT)
Name: Valor, dtype: object

In [101]:
with open("params.yaml") as config_file:
    config = yaml.safe_load(config_file)

xgb_p = config["train"]["estimators"]["xgboost"]["params"]

In [106]:
for k, v in xgb_p.items():
    print(k, v)

n_estimators 1000
learning_rate 0.5
max_depth 16
subsample 0.8
colsample_bytree 0.9
colsample_bylevel 0.4
scale_pos_weight 85


# Load Data

In [3]:
from src.stages.data_load import data_load

data_load("params.yaml")

2022-09-25 10:58:47,674 — DATA_LOAD — INFO — Reading data from SISAGUA (Datagov)
2022-09-25 10:59:04,591 — DATA_LOAD — INFO — Save data from SISAGUA (Datagov)
2022-09-25 10:59:04,792 — DATA_LOAD — INFO — Reading data from GEE
2022-09-25 11:04:20,425 — DATA_LOAD — INFO — Save data from GEE


In [5]:
from src.stages.data_label import data_label

data_label("params.yaml")

ParserError: while parsing a block mapping
  in "params.yaml", line 41, column 9
expected <block end>, but found ','
  in "params.yaml", line 45, column 32

In [17]:
from src.stages.data_split import data_split

data_split("params.yaml")


2022-08-23 15:46:35,086 — DATA_SPLIT — INFO — Load features
2022-08-23 15:46:35,095 — DATA_SPLIT — INFO — Split data into train and test
2022-08-23 15:46:35,098 — DATA_SPLIT — INFO — Oversampling training data


c:\Users\Daniel\cyanobacteria-monitoring\venv\lib\site-packages\smogn\over_sampling.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.iloc[:, j] = pd.Categorical(pd.factorize(
c:\Users\Daniel\cyanobacteria-monitoring\venv\lib\site-packages\smogn\over_sampling.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.iloc[:, j] = pd.Categorical(pd.factorize(
synth_matrix: 100%|##########| 6/6 [00:00<00:00, 36.09it/s]


2022-08-23 15:46:35,384 — DATA_SPLIT — INFO — Save train and test sets


In [12]:
with open("params.yaml") as config_file:
        config = yaml.safe_load(config_file)
    

df = pd.read_csv(config["data_load"]["labeled_df"])

In [41]:
from sklearn.model_selection import TimeSeriesSplit

df_train = df.loc[pd.to_datetime(df["date"]) <= "2022-01-28"].reset_index(drop=True)

tscv = TimeSeriesSplit(n_splits=3)
for train_index, val_index in tscv.split(df_train):
        df_train_2, df_val = df_train.iloc[train_index], df_train.iloc[val_index]

In [43]:
df_train_2

,date,NDVI_median,NDCI_median,B1_median,B2_median,B3_median,B4_median,B5_median,B6_median,B7_median,B8_median,B8A_median,B9_median,B11_median,B12_median,Resultado,Data da coleta,interval
0,2019-01-28,-0.396403,-0.034596,0.0419,0.0560,0.0794,0.0870,0.0830,0.0431,0.0422,0.0378,0.0322,0.0120,0.0245,0.0212,251,2019-01-28,9
1,2019-01-30,-0.523918,-0.040864,0.0267,0.0418,0.0617,0.0671,0.0618,0.0243,0.0269,0.0209,0.0169,0.0406,0.0137,0.0132,251,2019-01-28,9
2,2019-02-14,-0.700468,-0.101957,0.0263,0.0337,0.0592,0.0544,0.0443,0.0120,0.0122,0.0096,0.0066,0.0001,0.0053,0.0052,335,2019-02-11,12
3,2019-03-01,-0.775967,-0.138781,0.0224,0.0353,0.0557,0.0446,0.0336,0.0070,0.0086,0.0056,0.0026,0.0001,0.0032,0.0035,298,2019-02-25,15
4,2019-03-21,-0.608315,-0.103394,0.0353,0.0489,0.0744,0.0733,0.0595,0.0207,0.0224,0.0177,0.0125,0.0055,0.0086,0.0078,76,2019-03-18,19
5,2019-03-24,-0.614657,-0.095912,0.0285,0.0432,0.0675,0.0682,0.0562,0.0199,0.0212,0.0162,0.0122,0.0012,0.0055,0.0042,186,2019-03-26,20
6,2019-03-26,-0.596131,-0.088013,0.0397,0.0449,0.0702,0.0661,0.0555,0.0199,0.0199,0.0167,0.0122,0.0036,0.0076,0.0066,186,2019-03-26,20
7,2019-04-10,-0.674556,-0.117098,0.0235,0.0386,0.0609,0.0555,0.0435,0.0130,0.0145,0.0109,0.0079,0.0007,0.0051,0.0044,26,2019-04-09,23
8,2019-05-18,-0.584843,-0.047411,0.0317,0.0441,0.0770,0.0961,0.0870,0.0331,0.0324,0.0251,0.0174,0.0001,0.0059,0.0059,374,2019-05-20,31
9,2019-06-17,-0.639456,-0.061181,0.0468,0.0405,0.0482,0.0502,0.0445,0.0163,0.0147,0.0112,0.0085,0.0001,0.0055,0.0076,134,2019-06-17,37


In [45]:
get_ipython().system_raw()

TypeError: InteractiveShell.system_raw() missing 1 required positional argument: 'cmd'

In [4]:
from src.features.create_ratios import create_ratios

with open("params.yaml") as config_file:
        config = yaml.safe_load(config_file)
    
df_train = pd.read_csv(config["data_split"]["trainset_path"])
df_test = pd.read_csv(config["data_split"]["testset_path"])

df_train = create_ratios(df_train)
df_test = create_ratios(df_test)

features = config["featurize"]["selected_features"]

X_train = df_train[features]
X_test = df_test[features]
y_train = df_train[config["featurize"]["target_column"]]
y_test = df_test[config["featurize"]["target_column"]]


ModuleNotFoundError: No module named 'src.features.create_ratios'

# Train

## Ridge

In [48]:
# Poly Ridge



RANDOM = config["base"]["random_state"]

def objective(trial, X, y):

    param_grid = {
        "alpha": trial.suggest_float("alpha", .5, 100),
        "degree": trial.suggest_int("degree", 2, 5)
    }

    poly = PolynomialFeatures(degree=param_grid["degree"], include_bias=False)
    ridge_reg = Ridge(param_grid["alpha"])

    poly_ridge = Pipeline([
        ("poly", poly),
        ("regressor", ridge_reg)
    ])

    model = TransformedTargetRegressor(regressor = poly_ridge, transformer=PowerTransformer(method="yeo-johnson"))

    cv = RepeatedKFold(n_repeats=2, n_splits=3, random_state=RANDOM)

    cv_scores = []
    for (train_index, val_index) in cv.split(X, y):
    #print(train_index, val_index)
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]
        
        model.fit(X_train, y_train)
                
        preds = model.predict(X_val)
        cv_scores.append(mean_absolute_error(preds, y_val))
    
    return np.median(cv_scores)

study = optuna.create_study(direction="minimize", study_name="Ridge")
func = lambda trial: objective(trial, X_train, y_train)
study.optimize(func, n_trials=50)

[I 2022-08-23 17:01:26,467] A new study created in memory with name: Ridge
[I 2022-08-23 17:01:26,536] Trial 0 finished with value: 4352.396913628209 and parameters: {'alpha': 1.3221624241053715, 'degree': 3}. Best is trial 0 with value: 4352.396913628209.
[I 2022-08-23 17:01:26,603] Trial 1 finished with value: 4060.1721660702146 and parameters: {'alpha': 14.893994389666315, 'degree': 5}. Best is trial 1 with value: 4060.1721660702146.
[I 2022-08-23 17:01:26,655] Trial 2 finished with value: 4404.238905677783 and parameters: {'alpha': 38.93428750544299, 'degree': 2}. Best is trial 1 with value: 4060.1721660702146.
[I 2022-08-23 17:01:26,716] Trial 3 finished with value: 4438.45420266832 and parameters: {'alpha': 1.7726220386260472, 'degree': 2}. Best is trial 1 with value: 4060.1721660702146.
[I 2022-08-23 17:01:26,785] Trial 4 finished with value: 4370.8078660547035 and parameters: {'alpha': 1.522671159498604, 'degree': 3}. Best is trial 1 with value: 4060.1721660702146.
[I 2022-08-2

In [51]:
plot_optimization_history(study)

In [46]:
plot_param_importances(study)

In [52]:
study.best_params

{'alpha': 29.572716830739452, 'degree': 5}

In [75]:
poly = PolynomialFeatures(degree=3, include_bias=False)
ridge_reg = Ridge(alpha=30)

poly_ridge = Pipeline([
        ("poly", poly),
        ("regressor", ridge_reg)
    ])

model = TransformedTargetRegressor(regressor = poly_ridge, transformer=PowerTransformer(method="yeo-johnson"))

cross_val_score(model, X_train, y_train, cv=3, scoring="neg_mean_absolute_error")

array([-4159.22145147, -6818.81278819, -5402.80259775])

In [147]:
from sklearn.metrics import mean_absolute_percentage_error

RANDOM = 42

def objective_xgb(trial, X, y):

    param_grid = {
        "n_estimators": trial.suggest_categorical("n_estimators", [1000]),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 1),
        "min_child_weight": trial.suggest_int("min_child_weigth", 1, 10),
        "max_depth": trial.suggest_int("max_depth", 1, 50),
        "subsample": trial.suggest_float("subsample", .1, 1),
        "colsample_bytree": trial.suggest_float("colsample_bytree", .1, 1),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", .1, 1),
        "num_leaves": trial.suggest_int("num_leaves", 2**12, 2**16, step=20), 
        "reg_lambda": trial.suggest_int("lambda_l2", 1e-9, 100),
        "reg_alpha": trial.suggest_int("lambda_l1", 1e-9, 100),
        "gamma": trial.suggest_float("min_gain_to_split", 1e-9, .5),
        "scale_pos_weight": trial.suggest_float("scale_pos_weight", 1e-6, 500),
        "random_state": RANDOM
        }

    cv = KFold(n_splits=3, random_state=RANDOM, shuffle=True)
    cv_scores = []

    xgb = xgboost.XGBRegressor(**param_grid)

    poly = PolynomialFeatures(degree=3, include_bias=False)

    poly_reg = Pipeline([
        ("poly", poly),
        ("regressor", xgb)
    ])

    model = TransformedTargetRegressor(
        regressor = poly_reg,
        transformer=PowerTransformer(method="yeo-johnson")
        )
    
    for (train_index, val_index) in cv.split(X, y):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]    

        model.fit(
            X_train,
            y_train
        )

        preds = model.predict(X_val)
        cv_scores.append(mean_absolute_percentage_error(preds, y_val))
    
    return np.median(cv_scores)

study_xgb = optuna.create_study(direction="minimize", study_name="XGBoost Regression")
func_xgb = lambda trial: objective_xgb(trial, X_train, y_train)
study_xgb.optimize(func_xgb, n_trials=50)

[I 2022-08-27 18:11:08,302] A new study created in memory with name: XGBoost Regression
c:\Users\Daniel\cyanobacteria-monitoring\venv\lib\site-packages\optuna\distributions.py:560: UserWarning:

The distribution is specified by [1e-09, 100] and step=1, but the range is not divisible by `step`. It will be replaced by [1e-09, 99.000000001].



[18:11:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "num_leaves" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[18:11:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "num_leaves" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[18:11:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "num_leaves" } might not be used.

  This coul

[I 2022-08-27 18:11:11,804] Trial 0 finished with value: 1.8081622068266074 and parameters: {'n_estimators': 1000, 'learning_rate': 0.5244833451738737, 'min_child_weigth': 6, 'max_depth': 11, 'subsample': 0.1590678072629185, 'colsample_bytree': 0.9646018587786769, 'colsample_bylevel': 0.10142224171198755, 'num_leaves': 45016, 'lambda_l2': 13, 'lambda_l1': 59, 'min_gain_to_split': 0.06064324214888158, 'scale_pos_weight': 96.77069303534391}. Best is trial 0 with value: 1.8081622068266074.
c:\Users\Daniel\cyanobacteria-monitoring\venv\lib\site-packages\optuna\distributions.py:560: UserWarning:

The distribution is specified by [1e-09, 100] and step=1, but the range is not divisible by `step`. It will be replaced by [1e-09, 99.000000001].



[18:11:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "num_leaves" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[18:11:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "num_leaves" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[18:11:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "num_leaves" } might not be used.

  This coul

[I 2022-08-27 18:11:13,216] Trial 1 finished with value: 1.8081622068266074 and parameters: {'n_estimators': 1000, 'learning_rate': 0.7242830377148217, 'min_child_weigth': 4, 'max_depth': 21, 'subsample': 0.5107314111828498, 'colsample_bytree': 0.12751718979247625, 'colsample_bylevel': 0.1389399017618696, 'num_leaves': 41116, 'lambda_l2': 79, 'lambda_l1': 81, 'min_gain_to_split': 0.3740246801510778, 'scale_pos_weight': 55.384797315543956}. Best is trial 0 with value: 1.8081622068266074.
c:\Users\Daniel\cyanobacteria-monitoring\venv\lib\site-packages\optuna\distributions.py:560: UserWarning:

The distribution is specified by [1e-09, 100] and step=1, but the range is not divisible by `step`. It will be replaced by [1e-09, 99.000000001].



...

[I 2022-08-27 18:12:32,847] Trial 47 finished with value: 1.2740521031889995 and parameters: {'n_estimators': 1000, 'learning_rate': 0.5372497353608804, 'min_child_weigth': 6, 'max_depth': 22, 'subsample': 0.7621337007368627, 'colsample_bytree': 0.7232081846711338, 'colsample_bylevel': 0.5589616270924025, 'num_leaves': 20776, 'lambda_l2': 61, 'lambda_l1': 5, 'min_gain_to_split': 0.03568392128313911, 'scale_pos_weight': 112.24842236325338}. Best is trial 42 with value: 1.1470220166393397.
c:\Users\Daniel\cyanobacteria-monitoring\venv\lib\site-packages\optuna\distributions.py:560: UserWarning:

The distribution is specified by [1e-09, 100] and step=1, but the range is not divisible by `step`. It will be replaced by [1e-09, 99.000000001].



[18:12:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "num_leaves" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[18:12:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "num_leaves" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[18:12:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "num_leaves" } might not be used.

  This coul

[I 2022-08-27 18:12:34,592] Trial 48 finished with value: 1.1244175424875673 and parameters: {'n_estimators': 1000, 'learning_rate': 0.5289870001243008, 'min_child_weigth': 8, 'max_depth': 16, 'subsample': 0.7989955238950287, 'colsample_bytree': 0.940169248271379, 'colsample_bylevel': 0.40301969074782595, 'num_leaves': 35876, 'lambda_l2': 76, 'lambda_l1': 5, 'min_gain_to_split': 0.030750040500424965, 'scale_pos_weight': 85.35389688127495}. Best is trial 48 with value: 1.1244175424875673.
c:\Users\Daniel\cyanobacteria-monitoring\venv\lib\site-packages\optuna\distributions.py:560: UserWarning:

The distribution is specified by [1e-09, 100] and step=1, but the range is not divisible by `step`. It will be replaced by [1e-09, 99.000000001].



[18:12:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "num_leaves" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[18:12:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "num_leaves" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[18:12:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "num_leaves" } might not be used.

  This coul

[I 2022-08-27 18:12:36,225] Trial 49 finished with value: 2.0176808511194406 and parameters: {'n_estimators': 1000, 'learning_rate': 0.6663147101057207, 'min_child_weigth': 10, 'max_depth': 6, 'subsample': 0.8361320702600631, 'colsample_bytree': 0.9489048983138414, 'colsample_bylevel': 0.3155737127199245, 'num_leaves': 48056, 'lambda_l2': 86, 'lambda_l1': 12, 'min_gain_to_split': 0.022174008424312795, 'scale_pos_weight': 79.2167285968666}. Best is trial 48 with value: 1.1244175424875673.


In [148]:
plot_optimization_history(study_xgb)

In [149]:
plot_param_importances(study_xgb)

In [150]:
study_xgb.best_params

{'n_estimators': 1000,
 'learning_rate': 0.5289870001243008,
 'min_child_weigth': 8,
 'max_depth': 16,
 'subsample': 0.7989955238950287,
 'colsample_bytree': 0.940169248271379,
 'colsample_bylevel': 0.40301969074782595,
 'num_leaves': 35876,
 'lambda_l2': 76,
 'lambda_l1': 5,
 'min_gain_to_split': 0.030750040500424965,
 'scale_pos_weight': 85.35389688127495}

In [151]:
poly = PolynomialFeatures(degree=3, include_bias=False)

xgb_best_params = {
    'n_estimators': 1000,
    'learning_rate': 0.5,
    'max_depth': 16,
    'subsample': 0.8,
    'colsample_bytree': 0.9,
    'colsample_bylevel': 0.4,
    'scale_pos_weight': 85
}

reg = xgboost.XGBRegressor(**xgb_best_params)

poly_reg = Pipeline([
        ("poly", poly),
        ("regressor", reg)
    ])

model = TransformedTargetRegressor(
    regressor = poly_reg,
    transformer=PowerTransformer(method="yeo-johnson")
)

cross_val_score(model, X_train, y_train, cv=3, scoring="neg_mean_absolute_error")

array([-17086.41525252,   -478.63838412,  -2857.60446042])

In [152]:
import plotly.express as px
from sklearn.model_selection import train_test_split

X_train_train, X_train_val, y_train_train, y_train_val = train_test_split(
    X_train, y_train, test_size=.2, random_state=RANDOM
)

model.fit(X_train_train, y_train_train)
y_pred_val = model.predict(X_train_val)

fig = px.scatter(x = y_train_val, y = y_pred_val)
fig.update_layout(shapes=[{"type": "line", "y0":0, "y1":max(y_pred_val), "x0":0, "x1": max(y_pred_val)}])


In [128]:
X_train_train, X_train_val, y_train_train, y_train_val = train_test_split(
    X_train, y_train, test_size=.2, random_state=RANDOM
)

model.fit(X_train_train, y_train_train)
y_pred_val = model.predict(X_train_val)

fig = px.scatter(x = y_train_val, y = y_pred_val)
fig.update_layout(shapes=[{"type": "line", "y0":0, "y1":max(y_pred_val), "x0":0, "x1": max(y_pred_val)}])

In [16]:
def objective_rf(trial, X, y):

    param_grid = {
        "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2", None]),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10),
        "n_estimators": trial.suggest_int("n_estimators", 100, 2000)
        }

    cv = KFold(n_splits=3, random_state=42, shuffle=True)
    cv_scores = []

    rf = RandomForestRegressor(**param_grid, criterion="absolute_error")

    poly = PolynomialFeatures(degree=3, include_bias=False)

    poly_reg = Pipeline([
        ("poly", poly),
        ("regressor", rf)
    ])

    model = TransformedTargetRegressor(
        regressor = poly_reg,
        transformer=PowerTransformer(method="yeo-johnson")
        )
    
    for (train_index, val_index) in cv.split(X, y):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]    

        model.fit(
            X_train,
            y_train
        )

        preds = model.predict(X_val)
        cv_scores.append(mean_absolute_error(preds, y_val))
    
    return np.median(cv_scores)

study_rf = optuna.create_study(direction="minimize", study_name="Random Forest Regression")
func_rf = lambda trial: objective_rf(trial, X_train, y_train)
study_rf.optimize(func_rf, n_trials=50)

[I 2022-08-23 18:25:25,727] A new study created in memory with name: Random Forest Regression
[I 2022-08-23 18:25:41,108] Trial 0 finished with value: 2206.904739255515 and parameters: {'max_features': None, 'min_samples_leaf': 4, 'n_estimators': 1869}. Best is trial 0 with value: 2206.904739255515.
[I 2022-08-23 18:25:47,457] Trial 1 finished with value: 2378.816600496039 and parameters: {'max_features': 'sqrt', 'min_samples_leaf': 5, 'n_estimators': 1622}. Best is trial 0 with value: 2206.904739255515.
[I 2022-08-23 18:25:49,837] Trial 2 finished with value: 2941.439498822139 and parameters: {'max_features': 'sqrt', 'min_samples_leaf': 8, 'n_estimators': 620}. Best is trial 0 with value: 2206.904739255515.
[I 2022-08-23 18:25:59,129] Trial 3 finished with value: 1495.7780019428 and parameters: {'max_features': None, 'min_samples_leaf': 1, 'n_estimators': 723}. Best is trial 3 with value: 1495.7780019428.
[I 2022-08-23 18:26:00,951] Trial 4 finished with value: 1491.6432353063256 and 

In [25]:
study_rf.best_params

{'max_features': None, 'min_samples_leaf': 1, 'n_estimators': 615}

In [37]:
poly = PolynomialFeatures(degree=3, include_bias=False)

rf_best_params = {
    'max_features': "sqrt",
    'min_samples_leaf': 1,
    'n_estimators': 615
    }


reg = RandomForestRegressor(**rf_best_params, criterion="absolute_error")

poly_reg = Pipeline([
        ("poly", poly),
        ("regressor", reg)
    ])

model = TransformedTargetRegressor(
    regressor = poly_reg,
    transformer=PowerTransformer(method="yeo-johnson")
)

cross_val_score(model, X_train, y_train, cv=3, scoring="neg_mean_absolute_error")

array([-1855.54242931, -4206.29827522, -3154.08488835])

In [33]:
X_train_train, X_train_val, y_train_train, y_train_val = train_test_split(
    X_train, y_train, test_size=.2, random_state=42
)

model.fit(X_train_train, y_train_train)
y_pred_val = model.predict(X_train_val)

fig = px.scatter(x = y_train_val, y = y_pred_val)
fig.update_layout(shapes=[{"type": "line", "y0":0, "y1":max(y_pred_val), "x0":0, "x1": max(y_pred_val)}])

In [59]:
with open("params.yaml") as config_file:
        config = yaml.safe_load(config_file)

poly = PolynomialFeatures(degree=config["featurize"]["poly_degree"], include_bias=False)

xgb_params = config["train"]["estimators"]["xgboost"]["params"]
rf_params = config["train"]["estimators"]["random_forest"]["params"]
ridge_params = config["train"]["estimators"]["ridge"]["params"]

xgb_reg = xgboost.XGBRegressor(**xgb_params)
rf_reg = RandomForestRegressor(**rf_params)
ridge_reg = Ridge(**ridge_params)

def create_model(reg):
    poly_reg = Pipeline([
            ("poly", poly),
            ("regressor", reg)
        ])

    model = TransformedTargetRegressor(
        regressor = poly_reg,
        transformer=PowerTransformer(method="yeo-johnson")
    )

    return model

xgb_model = create_model(xgb_reg)
rf_model = create_model(rf_reg)
ridge_reg = create_model(ridge_reg)

In [62]:
xgb_model.fit(X_train, y_train)
rf_model.fit(X_train, y_train)
ridge_reg.fit(X_train, y_train)

TransformedTargetRegressor(regressor=Pipeline(steps=[('poly',
                                                      PolynomialFeatures(degree=3,
                                                                         include_bias=False)),
                                                     ('regressor',
                                                      Ridge(alpha=30))]),
                           transformer=PowerTransformer())

In [63]:
y_pred_xgb = xgb_model.predict(X_test)
y_pred_rf = rf_model.predict(X_test)
y_pred_ridge = ridge_reg.predict(X_test)

In [68]:
mean_absolute_error(y_test, y_pred_xgb)

4480.714920787811

In [73]:
y_pred = y_pred_ridge

fig = px.scatter(x = y_pred, y = y_test)
fig.update_layout(shapes=[{"type": "line", "y0":0, "y1":max(y_pred), "x0":0, "x1": max(y_pred)}])

In [93]:
max(max([1, 2]), max([2, 5]))

5

In [97]:
type(px.scatter(x=[1], y=[2]))

plotly.graph_objs._figure.Figure

In [ ]:
rf_params

reg = RandomForestRegressor(**rf_best_params, criterion="absolute_error")

poly_reg = Pipeline([
        ("poly", poly),
        ("regressor", reg)
    ])

model = TransformedTargetRegressor(
    regressor = poly_reg,
    transformer=PowerTransformer(method="yeo-johnson")
)

cross_val_score(model, X_train, y_train, cv=3, scoring="neg_mean_absolute_error")

In [86]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import RidgeCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Lasso

estimators = [
    ('ridge', ridge_reg),
    ('xgboost', xgb_reg),
    ('lasso', Lasso())
    #('knr', KNeighborsRegressor(n_neighbors=4, metric='minkowski'))
    ]

stack_reg = StackingRegressor(
    estimators = estimators,
    final_estimator=RidgeCV()
)

stack_model = create_model(stack_reg)

cross_val_score(stack_model, X_train, y_train, cv=3, scoring="neg_mean_absolute_error")

array([-3545.84301486, -4014.6469351 , -1942.49528751])

In [87]:
stack_model.fit(X_train, y_train)
y_stack = stack_model.predict(X_test)
mean_absolute_error(y_test, y_stack)

4699.349422840397

In [88]:
y_pred = y_stack

fig = px.scatter(x = y_pred, y = y_test)
fig.update_layout(shapes=[{"type": "line", "y0":0, "y1":max(y_pred), "x0":0, "x1": max(y_pred)}])

In [89]:
from sklearn.ensemble import VotingRegressor

vr = VotingRegressor(estimators=estimators)

voting_model = create_model(vr)

cross_val_score(voting_model, X_train, y_train, cv=3, scoring="neg_mean_absolute_error")

array([-3930.46262618, -4177.41226852, -2474.57205878])

In [90]:
voting_model.fit(X_train, y_train)
y_voting = stack_model.predict(X_test)
print(mean_absolute_error(y_test, y_voting))

y_pred = y_voting

fig = px.scatter(x = y_pred, y = y_test)
fig.update_layout(shapes=[{"type": "line", "y0":0, "y1":max(y_pred), "x0":0, "x1": max(y_pred)}])

4699.349422840397


In [116]:
with open("params.yaml") as config_file:
    config = yaml.safe_load(config_file)

pred = pd.read_csv(config["evaluate"]["final_predictions_file"])

pred.head()

,date,y_pred
0,2018-12-24 13:40:42,316.016800
1,2018-12-26 13:30:43,47.188125
2,2018-12-29 13:40:40,311.473424
3,2019-01-25 13:30:47,72.442495
4,2019-01-28 13:40:43,220.278946


In [123]:
from src.data.label_gee import load_data

_, ciano = load_data("params.yaml")

2022-08-27 13:50:41,454 — DATA_LABEL — INFO — Reading data
2022-08-27 13:50:41,505 — DATA_LABEL — INFO — Filtering data


c:\Users\Daniel\cyanobacteria-monitoring\src\data\label_gee.py:30: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\Users\Daniel\cyanobacteria-monitoring\src\data\label_gee.py:31: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [126]:
ciano.head()

,Data da coleta,Resultado
4873,2018-12-19,719
2949,2019-01-18,0
2538,2019-01-28,251
4881,2019-02-11,335
3723,2019-02-25,298


In [154]:
#https://stackoverflow.com/questions/62287001/how-to-overlay-two-plots-in-same-figure-in-plotly-create-pareto-chart-in-plotl


import plotly.graph_objects as go
from plotly.subplots import make_subplots

gee_plot = go.Scatter(
    x=pred["date"],
    y=pred["y_pred"],
    name="Predicted values")

vigi_plot = go.Scatter(
    x=ciano["Data da coleta"],
    y=ciano["Resultado"],
    mode="markers",
    name = "SISAGUA")

fig = make_subplots()
fig.add_trace(gee_plot)
fig.add_trace(vigi_plot)

#fig.add_trace(px.scatter(ciano, x="Data da coleta", y = "Resultado"))

In [156]:
from src.report.plot_predicted_values import plot_predicted_values

plot_predicted_values("params.yaml")

2022-08-27 18:46:35,583 — DATA_LABEL — INFO — Reading data
2022-08-27 18:46:35,643 — DATA_LABEL — INFO — Filtering data


c:\Users\Daniel\cyanobacteria-monitoring\src\data\label_gee.py:30: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\Users\Daniel\cyanobacteria-monitoring\src\data\label_gee.py:31: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [12]:
import yaml

with open("params.yaml") as config_file:
    config = yaml.safe_load(config_file)

In [9]:
x = config["featurize"]["selected_clean_columns"]

x + ['Resultado', 'Data da coleta', 'interval']

['date',
 'NDVI_median',
 'NDCI_median',
 'B1_median',
 'B2_median',
 'B3_median',
 'B4_median',
 'B5_median',
 'B6_median',
 'B7_median',
 'B8_median',
 'B8A_median',
 'B9_median',
 'B11_median',
 'B12_median',
 ['Resultado', 'Data da coleta', 'interval'],
 ['Resultado', 'Data da coleta', 'interval'],
 ['Resultado', 'Data da coleta', 'interval'],
 'Resultado',
 'Data da coleta',
 'interval']

In [14]:
selected_columns = config["featurize"]["selected_clean_columns"] 
selected_columns

['date',
 'NDVI_median',
 'NDCI_median',
 'B1_median',
 'B2_median',
 'B3_median',
 'B4_median',
 'B5_median',
 'B6_median',
 'B7_median',
 'B8_median',
 'B8A_median',
 'B9_median',
 'B11_median',
 'B12_median',
 'Resultado',
 'Data da coleta',
 'interval',
 'Resultado',
 'Data da coleta',
 'interval']